In [ ]:
!pip install climetlab climetlab-eumetnet-postprocessing-benchmark xarray pandas matplotlib numpy zarr

In [ ]:
!pip install zarr

In [4]:
import numpy as np
import climetlab as cml
import xarray as xr 
import pandas as pd
import os
import zarr

In [ ]:
# Open the Zarr store with consolidated metadata
ds = xr.open_zarr("saved_variables.zarr", consolidated=True)

# Print metadata
print(ds)

# Access individual variables
t2m = ds["t2m"]
u10 = ds["u10"]
v10 = ds["v10"]
tcc = ds["tcc"]
u100 = ds["u100"]
v100 = ds["v100"]
z = ds["z"]
u700 = ds["u700"]
v700 = ds["v700"]
t = ds["t"]
p10fg6 = ds["p10fg6"]
ssr6 = ds["ssr6"]
ssrd6 = ds["ssrd6"]

In [ ]:
# Open the entire dataset from the Zarr store
ds = xr.open_zarr("full_datasets.zarr", consolidated=True)

# Print dataset metadata to confirm loading
print(ds)

In [5]:
# #### Surface variables 

sfc_rfc =cml.load_dataset('EUPPBench-training-data-gridded-reforecasts-surface')

By downloading data from this dataset, you agree to the terms and conditions defined at

    https://github.com/Climdyn/climetlab-eumetnet-postprocessing-benchmark/blob/main/DATA_LICENSE

If you do not agree with such terms, do not download the data. 


/home/jupyter-ayoub/.local/lib/python3.10/site-packages/climetlab/sources/zarr.py:92: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  self._ds = xr.open_dataset(store, engine="zarr")  # TODO: chunks="auto" ?


In [6]:
# #### presure level variables 

pl500_rfc = cml.load_dataset('EUPPBench-training-data-gridded-reforecasts-pressure', level='500')
pl700_rfc = cml.load_dataset('EUPPBench-training-data-gridded-reforecasts-pressure', level='700')
pl850_rfc=cml.load_dataset('EUPPBench-training-data-gridded-reforecasts-pressure', level='850')

By downloading data from this dataset, you agree to the terms and conditions defined at

    https://github.com/Climdyn/climetlab-eumetnet-postprocessing-benchmark/blob/main/DATA_LICENSE

If you do not agree with such terms, do not download the data. 


/home/jupyter-ayoub/.local/lib/python3.10/site-packages/climetlab/sources/zarr.py:92: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  self._ds = xr.open_dataset(store, engine="zarr")  # TODO: chunks="auto" ?
/home/jupyter-ayoub/.local/lib/python3.10/site-packages/climetlab/sources/zarr.py:92: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  self._ds = xr.open_dataset(store, engine="zarr")  # TODO: chunks="auto" ?
/home/jupyter-ayoub/.local/lib/python3.10/site-packages/climetlab/sources/zarr.py:92: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instan

In [7]:
# #### processed variables

proc_rfc = cml.load_dataset('EUPPBench-training-data-gridded-reforecasts-surface-processed')

By downloading data from this dataset, you agree to the terms and conditions defined at

    https://github.com/Climdyn/climetlab-eumetnet-postprocessing-benchmark/blob/main/DATA_LICENSE

If you do not agree with such terms, do not download the data. 


/home/jupyter-ayoub/.local/lib/python3.10/site-packages/climetlab/sources/zarr.py:92: FutureWarning: In a future version of xarray decode_timedelta will default to False rather than None. To silence this warning, set decode_timedelta to True, False, or a 'CFTimedeltaCoder' instance.
  self._ds = xr.open_dataset(store, engine="zarr")  # TODO: chunks="auto" ?


In [ ]:
sfc_rfc.to_xarray().to_zarr("full_datasets.zarr", mode="w")
pl500_rfc.to_xarray().to_zarr("full_datasets.zarr", mode="a")
pl700_rfc.to_xarray().to_zarr("full_datasets.zarr", mode="a")
pl850_rfc.to_xarray().to_zarr("full_datasets.zarr", mode="a")
proc_rfc.to_xarray().to_zarr("full_datasets.zarr", mode="a")

# **Consolidate metadata** after all datasets are written
zarr.consolidate_metadata("full_datasets.zarr")

In [8]:
t2m=sfc_rfc.to_xarray()[['t2m']] 
u10=sfc_rfc.to_xarray()[['u10']]
v10=sfc_rfc.to_xarray()[['v10']]
tcc=sfc_rfc.to_xarray()[['tcc']]
u100 =sfc_rfc.to_xarray()[['u100']]
v100 =sfc_rfc.to_xarray()[['v100']]

In [9]:
z=pl500_rfc.to_xarray()[['z']] 
u700=pl700_rfc.to_xarray()[['u']]
v700= pl700_rfc.to_xarray()[['v']]
t=pl850_rfc.to_xarray()[['t']]

In [10]:
p10fg6=proc_rfc.to_xarray()[['p10fg6']]
ssr6=proc_rfc.to_xarray()[['ssr6']]
ssrd6=proc_rfc.to_xarray()[['ssrd6']]

In [8]:
print(type(u10['u10'].data))  # DaskArray of NumpyArray?

<class 'numpy.ndarray'>


In [ ]:
# Put them in a list
datasets = [
    t2m, u10, v10, tcc, u100, v100,
    z, u700, v700, t,
    p10fg6, ssr6, ssrd6
]

In [ ]:
# Path to save
zarr_path = "saved_variables.zarr"

In [ ]:
def create_encoding(ds):
    return {
        var: {
            "chunks": (10, 100, 100),  # gebruik "chunks" ipv "chunksizes"
            "compressor": zarr.Blosc(cname="zstd", clevel=3)
        }
        for var in ds.data_vars
    }


In [ ]:
# Schrijf datasets met encoding
for i, ds in enumerate(datasets):
    mode = "w" if i == 0 else "a"
    encoding = create_encoding(ds)
    ds.to_zarr(zarr_path, mode=mode, encoding=encoding )

In [ ]:
# Write the first dataset with mode="w", the rest with mode="a"
for i, ds in enumerate(datasets):
    mode = "w" if i == 0 else "a"
    ds.to_zarr(zarr_path, mode=mode, decode_timedelta=True)

In [ ]:
# Consolidate metadata
zarr.consolidate_metadata(zarr_path)

In [11]:
w10_calc = np.sqrt(u10['u10']**2 + v10['v10']**2)
w10 = xr.Dataset({'w10': w10_calc})

In [1]:
w100_calc = np.sqrt(u100['u100']**2 + v100['v100']**2)
w100 = xr.Dataset({'w100': w100_calc})

NameError: name 'np' is not defined

In [ ]:
w700_calc =  np.sqrt(u700['u']**2 + v700['v']**2)
w700 = xr.Dataset({'w700': w700_calc})

In [ ]:
# #### Getting the observations 
u100_obs=sfc_rfc.get_observations_as_xarray()[['u100']]
v100_obs=sfc_rfc.get_observations_as_xarray()[['v100']]

In [ ]:
w100_obs_calc= np.sqrt(u100_obs['u100']**2 + v100_obs['v100']**2)

In [ ]:
w100_obs_DA = xr.DataArray(
    w100_obs_calc,
    dims=('time', 'number', 'year', 'step', 'surface', 'latitude', 'longitude'),
    coords=u100_obs.coords,
    name='w100_obs'
)

In [ ]:
# Now create the Dataset with `w100_obs`
w100_obs = xr.Dataset({'w100_obs': w100_obs_DA})

In [ ]:
#some preprocessing
datasets = [t2m, u10, w10, tcc, u100, w100,z, u700,w700, t,p10fg6,w100_obs,ssr6,ssrd6] 
for i in range(len(datasets)):
    # Convert 'step' from nanoseconds to hours
    datasets[i]['step'] = pd.to_timedelta(datasets[i]['step'], unit='ns').total_seconds() / 3600
    # Select specific time steps
    if i!=10:  #remove first forecasting step because it is absent in the processed variables 
         datasets[i] = datasets[i].isel(step=slice(1,None))
    # Squeeze unnecessary dimensions
    if 'depthBelowLandLayer' in datasets[i].dims:
        datasets[i] = datasets[i].squeeze('depthBelowLandLayer')
    if 'surface' in datasets[i].dims:
        datasets[i] = datasets[i].squeeze('surface')
    if 'isobaricInhPa' in datasets[i].variables:
        datasets[i] = datasets[i].drop_vars('isobaricInhPa')
    if "isobaricInhPa" in datasets[i].dims:
        datasets[i] = datasets[i].squeeze(dim="isobaricInhPa")
    if 'surface' in datasets[i].variables:
        datasets[i] = datasets[i].drop_vars('surface')
    if "surface" in datasets[i].dims:
        datasets[i] = datasets[i].squeeze(dim="surface")

In [ ]:
#some preprocessing
datasets = [t2m, u10, w10, v10, tcc, u100, w100, v100, z, u700, w700, t, p10fg6, w100_obs, ssr6, ssrd6]
for i in range(len(datasets)):
    # Convert 'step' from nanoseconds to hours
    datasets[i]['step'] = pd.to_timedelta(datasets[i]['step'], unit='ns').total_seconds() / 3600
    # Select specific time steps
    if i!=10:  #remove first forecasting step because it is absent in the processed variables 
         datasets[i] = datasets[i].isel(step=slice(1,None))
    # Squeeze unnecessary dimensions
    if 'depthBelowLandLayer' in datasets[i].dims:
        datasets[i] = datasets[i].squeeze('depthBelowLandLayer')
    if 'surface' in datasets[i].dims:
        datasets[i] = datasets[i].squeeze('surface')
    if 'isobaricInhPa' in datasets[i].variables:
        datasets[i] = datasets[i].drop_vars('isobaricInhPa')
    if "isobaricInhPa" in datasets[i].dims:
        datasets[i] = datasets[i].squeeze(dim="isobaricInhPa")
    if 'surface' in datasets[i].variables:
        datasets[i] = datasets[i].drop_vars('surface')
    if "surface" in datasets[i].dims:
        datasets[i] = datasets[i].squeeze(dim="surface")

In [ ]:
rfc_all = xr.merge(datasets[0:17], compat='override')

In [ ]:
output_dir_forecast = "./EUPP"
output_dir_era5 ="./ERA5"

In [ ]:
# Give rights
!chmod 777 /home/jupyter-ayoub/data/EUPP
!chmod 777 /home/jupyter-ayoub/data/ERA5

In [ ]:
#save files in format fitted to the loader 

for year in range(1):
    yeardata=rfc_all.isel(year=year)
    for time in yeardata.time:
        time_str = pd.to_datetime(str(time.values)).strftime('%Y%m%d')
        filename = f"output.sfc.{int(year)}.{time_str}.nc"
        filepath = os.path.join(output_dir_forecast, filename)
        # Select the data for the current time step
        xds_at_time=yeardata.sel(time=time)
        xds_at_time.to_netcdf(filepath)

In [ ]:
#observations
for year in range(1): 
    yeardata=w100_obs.isel(year=year)
    for time in yeardata.time:
        time_str = pd.to_datetime(str(time.values)).strftime('%Y%m%d')
        filename = f"era.sfc.{int(year)}.{time_str}.nc"
        filepath = os.path.join(output_dir_era5, filename)
        # Select the data for the current time step
        xds_at_time=yeardata.sel(time=time)
        xds_at_time.to_netcdf(filepath)

In [ ]:
# Verkrijg alle variabelen in de dataset
variable_names = list(rfc_all.variables)

# Print enkel de namen van de variabelen
variable_names